In [ ]:
!pip install torch
!pip install git+https://github.com/huggingface/transformers
!pip install bnlp_toolkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-9328e8a5
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-9328e8a5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163 kB 31.2 MB/s 
     |████████████████████████████████| 7.6 MB 40.7 MB/s 
  Created wheel for transformers: filename=transformers-4.25.0.dev0-py3-none-any.whl size=5665671 sha256=c426b80ca0be0c26accf692da7785ddc2f0a3e6acf7ebb9a6805a8458627d247
  Stored in directory: /tmp/pip-ephem-wheel-cache-l74fqqiw/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
Looking in indexes: https://pypi.org/simple,

In [ ]:
# download = drive.CreateFile({'id': '1h17UOFA407GDi0YJcKbnNuXFkg5rZ8TV'})
# download.GetContentFile('final_large_data.csv')
# # https://drive.google.com/file/d/1h17UOFA407GDi0YJcKbnNuXFkg5rZ8TV/view?usp=sharing

In [ ]:
import time

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from tqdm.notebook import tqdm
from transformers import BertModel, BertTokenizer, BertForSequenceClassification

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd

In [ ]:
def analysis(data):
    missing_percent = data.isnull().sum() * 100 / len(data)
    total_missing = data.isnull().sum()
    unique = data.nunique()
    not_null = data.notnull().sum()
    df = pd.DataFrame({'Unique Values': unique,
                        'Value Exist': not_null,
                        'Total_Missing': total_missing,
                         'Missing %': missing_percent})
    print('Data Shape: ',data.shape)
    df = df.round(2)
    return df

In [ ]:
dataset = pd.read_csv('final_large_data.csv')

In [ ]:
def generate_label_id_from_name(df):
    label_names = df['label'].unique().tolist()
    lenght_label_names = len(label_names)
    label_maps = dict(zip(label_names, [i for i in range(0, lenght_label_names)]))
    label_name_with_id_df = pd.DataFrame.from_dict(label_maps.items())
    label_name_with_id_df.columns = ['label', 'label_id']
    return label_name_with_id_df

In [ ]:
def add_label_id(df):
    if 'label_id' in df.columns:
        return df
    else:
        label_df = generate_label_id_from_name(df)
        dfm = df.merge(label_df, on="label", how="left")
        return dfm

In [ ]:
concated_dataset = dataset.copy()
concated_dataset = add_label_id(concated_dataset)
concated_dataset.dropna(inplace=True)
concated_dataset = concated_dataset.rename(columns={
        "label": "NewsType"
    })
concated_dataset = concated_dataset.rename(columns={
        "label_id": "label"
    })
analysis(concated_dataset)

Data Shape:  (408428, 3)


,Unique Values,Value Exist,Total_Missing,Missing %
status_text,406878,408428,0,0.0
NewsType,9,408428,0,0.0
label,9,408428,0,0.0


In [ ]:
possible_labels = concated_dataset.NewsType.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'bangladesh': 0,
 'sports': 1,
 'technology': 2,
 'entertainment': 3,
 'international': 4,
 'economy': 5,
 'life-style': 6,
 'opinion': 7,
 'education': 8}

In [ ]:
df = concated_dataset.copy()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.2, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
df.head(3)

,status_text,NewsType,label,data_type
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,bangladesh,0,train
1,এবারের উইম্বলডনটা স্মরণীয় করে রাখার মিশনেই যে...,sports,1,train
2,জাতীয় সংসদে বিএনপি চেয়ারপারসন ও বিরোধীদলীয় ...,bangladesh,0,train


In [ ]:

df.groupby(['NewsType', 'label', 'data_type']).count()

status_text
NewsType      label data_type             
bangladesh    0     train           186000
                    val              46500
economy       5     train            13796
                    val               3449
education     8     train             7777
                    val               1944
entertainment 3     train            24369
                    val               6092
international 4     train            24684
                    val               6171
life-style    6     train             8665
                    val               2166
opinion       7     train            12559
                    val               3140
sports        1     train            39201
                    val               9801
technology    2     train             9691
                    val               2423

In [ ]:
tokenizer = BertTokenizer.from_pretrained('sagorsarker/bangla-bert-base', 
                                          do_lower_case=True)

Downloading:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/491 [00:00<?, ?B/s]

In [ ]:
df = df.rename(columns={
        "status_text": "Headline"
    })

In [ ]:
%%time
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Headline.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True, 
    max_length=128,
    truncation=True,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Headline.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True, 
    max_length=128,
    truncation=True,
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

CPU times: user 54min 36s, sys: 9.69 s, total: 54min 46s
Wall time: 54min 42s


In [ ]:
from torch.utils.data import TensorDataset

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
len(dataset_train), len(dataset_val)

(326742, 81686)

In [ ]:
model = BertForSequenceClassification.from_pretrained("sagorsarker/bangla-bert-base",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Downloading:   0%|          | 0.00/660M [00:00<?, ?B/s]

Some weights of the model checkpoint at sagorsarker/bangla-bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 65

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [ ]:
epochs = 1

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random
import numpy as np
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
# def evaluate_input(dataloader_input):

#     model.eval()
    
#     loss_val_total = 0
#     predictions = []
    
#     for batch in dataloader_input:
#         # print('before process batch-->', batch)
#         batch = tuple(b.to(device) for b in batch)
#         # print('after process batch-->', batch)

#         inputs = {'input_ids':      batch[0],
#                   'attention_mask': batch[1],
#                  }

#         with torch.no_grad():        
#             outputs = model(**inputs)
            
#         # print('outputs')
#         # print(outputs)
#         loss = outputs[0]
#         # print('loss')
#         # print(loss)
#         _, prediction = torch.max(outputs[0], dim=1)
#         # print(prediction)
#         # logits = outputs[1]
#         # print('logits')
#         # print(logits)
#         # loss_val_total += loss.item()
#         # print('loss_val_total')
#         # print(loss_val_total)
#         # logits = logits.detach().cpu().numpy()
#         # print('logits detach')
#         # print(logits)
#         # # label_ids = inputs['labels'].cpu().numpy()
#         predictions.append(prediction)
#         # true_vals.append(label_ids)
    
#     # loss_val_avg = loss_val_total/len(dataloader_input) 
    
#     # predictions = np.concatenate(predictions, axis=0)
#     print('... pred....')
#     print(predictions)
#     # true_vals = np.concatenate(true_vals, axis=0)
            
#     return predictions

In [ ]:
%%time
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'./head_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/5027 [00:00<?, ?it/s]

In [ ]:
model

In [ ]:
model = BertForSequenceClassification.from_pretrained("sagorsarker/bangla-bert-base",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# input_text = "বাংলাদেশ ব্যাংকের বৈদেশিক মুদ্রার রিজার্ভ কমে ৩ হাজার ৪০০ কোটি ডলারের ঘরে নেমে এসেছে।"

# encoded_input_data = tokenizer.batch_encode_plus(
#     input_text, 
#     add_special_tokens=True, 
#     return_attention_mask=True, 
#     pad_to_max_length=True, 
#     return_token_type_ids=False,
#     max_length=256, 
#     return_tensors='pt'
# )

# input_ids = encoded_input_data['input_ids'].to(device)
# attention_masks = encoded_input_data['attention_mask'].to(device)
# output = model(input_ids, attention_masks)
# _, prediction = torch.max(output[0], dim=1)
# print(f'Review text: {input_text}')
# print(prediction)
# # print(f'Sentiment  : {class_names[prediction]}')
# # return class_names[prediction]


In [ ]:
# input_text = "বাংলাদেশ ব্যাংকের বৈদেশিক মুদ্রার রিজার্ভ কমে ৩ হাজার ৪০০ কোটি ডলারের ঘরে নেমে এসেছে।"

# encoded_input_data = tokenizer.batch_encode_plus(
#     input_text, 
#     add_special_tokens=True, 
#     return_attention_mask=True, 
#     pad_to_max_length=True, 
#     return_token_type_ids=False,
#     max_length=256, 
#     return_tensors='pt'
# )

# input_ids = encoded_input_data['input_ids']
# attention_masks = encoded_input_data['attention_mask']

# dataset_input = TensorDataset(input_ids, attention_masks)

# dataloader_input = DataLoader(dataset_input, 
#                                    sampler=SequentialSampler(dataset_input), 
#                                    batch_size=batch_size)


# prediction = evaluate_input(dataloader_input)

# # print('prediction')
# # print(prediction)

In [ ]:
torch.save(model, '/content/final_model')

In [ ]:
model = BertForSequenceClassification.from_pretrained("sagorsarker/bangla-bert-base",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

In [ ]:
model.load_state_dict(torch.load('./head_5.model', map_location=torch.device('cpu')))

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)

In [ ]:
# train_loss_data, valid_loss_data = [], []
# valid_loss_min = np.Inf
# since = time.time()
# best_loss = np.inf

# for epoch in range(epochs):
#     print("Epoch: {}/{}".format(epoch + 1, epochs))
#     # monitor training loss
#     train_loss = 0.0
#     valid_loss = 0.0
#     total = 0
#     correct = 0
#     e_since = time.time()

#     # Train Model
#     train_loss += train(model, train_dataloader, optimizer, criterion, tokenizer_config)
#     # Now Evaluate
#     out = evaluate(model, test_dataloader, criterion, tokenizer_config)
#     total += out[0]
#     correct += out[1]
#     valid_loss += out[2]
    
#     scheduler.step()

#     # print training/validation statistics
#     # calculate average loss over an epoch
#     train_loss = train_loss / len(train_dataloader.dataset)
#     valid_loss = valid_loss / len(test_dataloader.dataset)

#     # calculate train loss and running loss
#     train_loss_data.append(train_loss * 100)
#     valid_loss_data.append(valid_loss * 100)
    
#     if valid_loss < best_loss:
#         best_loss = valid_loss
#         torch.save(model.state_dict(), "news_model1.pth")

#     print("\tTrain loss:{:.6f}..".format(train_loss),
#           "\tValid Loss:{:.6f}..".format(valid_loss),
#           "\tAccuracy: {:.4f}".format(correct / total * 100))

# time_elapsed = time.time() - since
# print('Training completed in {:.0f}m {:.0f}s'.format(
#     time_elapsed // 60, time_elapsed % 60))